# Web scraping serie de Dólar Blue

El código scrapea el dato de dólar blue de la página de de La Voz del Interior y lo almacena en un excel (serie_dolar.xlsx).
Es necesario correrlo todos los días para agregar un nuevo valor a la serie.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import date

In [2]:
# Obtener el contenido de la página y ubicar el valor del cuarto div (si no, "Error")
try:
    page = requests.get('https://www.lavoz.com.ar/')
    soup = BeautifulSoup(page.content, 'html.parser')
    cuarto_div = soup.find_all("div", class_="dollar-value")[3]
    p_tag = cuarto_div.find("p", class_="value")
    tags_list = list(p_tag.children)
    valor = float(tags_list[2].strip('$').replace(',', '.'))
except (IndexError, AttributeError, ValueError, requests.exceptions.RequestException):
    valor = "Error"

In [3]:
# Crear un nuevo DataFrame con la fecha de hoy más el valor
fecha = date.today().strftime('%d/%m/%Y')
nuevo_renglon = pd.DataFrame({'Fecha': [fecha], 'Valor': [valor]})

# Leer un Excel existente y almacenarlo en un DataFrame
df = pd.read_excel('serie_dolar.xlsx')

# Unir ambos DataFrames
df = pd.concat([df, nuevo_renglon], ignore_index=True)

# Guardar el resultado en el mismo Excel importado
writer = pd.ExcelWriter("serie_dolar.xlsx", engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()